Laboratorio 1 - Regresión lineal 
Integrantes: 
Silvana Echeverry - 202310470
David Mora - 
Carlos Vargas

In [2]:
#importando librerias 

import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

In [9]:
data_train = pd.read_csv('/Users/silvanaecheverry/Downloads/datos_entrenamiento_laboratorio1(train_data).csv')
data_test = pd.read_csv('/Users/silvanaecheverry/Downloads/datos_validacion_laboratorio1(test_data).csv')

1. Entendimiento de los datos 

Para esta primera etapa, decidimos hacer uso de diferentes funciones para entender como se comportaban nuestros datos, como: 
* Head() -> muestra las primera filas del data frame con las columnas 
* isna().sum() -> nos permitio contar los datos que tenian valores nulos,
* dtypes -> saber los tipos de las variables que tenemos

Despues de hacer esta exploración de los datos, nos dimos cuenta que ciertas variables importantes estaban con valores nulos, o variables importantes estaban con valores que no les correspondian, por ejemplo: 
- edad: object -> y deberia ser int
- market value: object -> y deberia ser int
- dia de partido: object -> fecha 
y con los valores nulos: 
- Edad 48
- Nacionalidad 32
- %_de_regates_exitosos 28.722
- %_de_duelos_aereos_ganados 20.957
- market_value 12.894 (solo esta en el archivo de entrenamiento)
- contract_date 12.916

Nuestra variable objetivo es predecir el valor de mercado de un jugador (market_value) a partir de sus estadísticas registradas durante partidos. 

Cada fila del data set representa el desempeño de un jugador en un partido especifico (fecha de partido). Nos podemos dar cuenta que hay multiples registros de cada jugador en cada uno de los partidos. 

Además, encontramos que ciertas columnas estan repetidas con datos duplicados como: 
- xAG
- Pasos_Progresivos
- Regates_Exitosos
- Pases_completados

Para hacer una correcta estimación, en esta etapa consideramos que hacen falta variables en cuenta para la estimación, como: 
- partidos_jugados: Cantidad total de partidos registrados por jugador en la temporada
- valor_por_partido: Valor de mercado dividido entre partidos_jugados, ya que segun el partido pueden tener diferente valor
- posicion_general: una variable que categorice la posición del jugador



In [11]:
' Usamos estas lineas de codigo para explorar los datos '
#data_test.head()
#data_train.head()
#data_train.isna().sum() #verificando datos nulos
#data_test.isna().sum()
#data_test.dtypes
#data_train.dtypes

' Usamos estas lineas de codigo para explorar los datos '